In [ ]:
!pip install allennlp==1.0.0
!pip install allennlp-models==1.0.0
!git clone https://github.com/mhagiwara/realworldnlp.git
%cd realworldnlp

In [ ]:
from typing import Dict

import numpy as np
import torch
import torch.optim as optim

# from allennlp.data.dataset_readers import UniversalDependenciesDatasetReader
from allennlp_models.structured_prediction.dataset_readers.universal_dependencies import UniversalDependenciesDatasetReader
# from allennlp.data.iterators import BucketIterator
from allennlp.data import DataLoader
from allennlp.data.samplers import BucketBatchSampler

from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
# from allennlp.training.trainer import Trainer
from allennlp.training.trainer import GradientDescentTrainer

from realworldnlp.predictors import UniversalPOSPredictor

In [ ]:
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 128

In [ ]:
class LstmTagger(Model):
    def __init__(self,
                 embedder: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.embedder = embedder
        self.encoder = encoder
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('pos'))
        self.accuracy = CategoricalAccuracy()

    def forward(self,
                words: Dict[str, torch.Tensor],
                pos_tags: torch.Tensor = None,
                **args) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(words)

        embeddings = self.embedder(words)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.linear(encoder_out)

        output = {"tag_logits": tag_logits}
        if pos_tags is not None:
            self.accuracy(tag_logits, pos_tags, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, pos_tags, mask)

        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [ ]:
reader = UniversalDependenciesDatasetReader()

In [ ]:
train_dataset = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/ud-treebanks-v2.3/UD_English-EWT/en_ewt-ud-train.conllu')
dev_dataset = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/ud-treebanks-v2.3/UD_English-EWT/en_ewt-ud-dev.conllu')

In [ ]:
vocab = Vocabulary.from_instances(train_dataset + dev_dataset)

In [ ]:
train_dataset.index_with(vocab)
dev_dataset.index_with(vocab)

In [ ]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_SIZE)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [ ]:
encoder = PytorchSeq2SeqWrapper(
    torch.nn.LSTM(EMBEDDING_SIZE, HIDDEN_SIZE, batch_first=True))

In [ ]:
model = LstmTagger(word_embeddings, encoder, vocab)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
# iterator = BucketIterator(batch_size=16, sorting_keys=[("words", "num_tokens")])
train_data_loader = DataLoader(
    train_dataset,
    batch_sampler=BucketBatchSampler(
        train_dataset,
        batch_size=32,
        sorting_keys=["words"]
    )
)

dev_data_loader = DataLoader(
    train_dataset,
    batch_sampler=BucketBatchSampler(
        dev_dataset,
        batch_size=32,
        sorting_keys=["words"]
    )
)

In [ ]:
# iterator.index_with(vocab)

In [ ]:
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=dev_data_loader,
    patience=10,
    num_epochs=10)
trainer.train()

In [ ]:
predictor = UniversalPOSPredictor(model, reader)
tokens = ['The', 'dog', 'ate', 'the', 'apple', '.']
logits = predictor.predict(tokens)['tag_logits']
tag_ids = np.argmax(logits, axis=-1)

[vocab.get_token_from_index(tag_id, 'pos') for tag_id in tag_ids]